# Example reading fits file

Here we will read a fits file using astropy, a very common module used by many astronomers.

First we need to import the modules in python we need.


In [ ]:
from astropy.io import fits
import numpy as np
import math

Now we are ready to open the file.  We need to know that a FITS file consists of one or more "header-data-units (hdu)"


In [ ]:
filename = 'test3_0652.fits'

In [ ]:
hdu = fits.open(filename,ignore_missing_end=True)
print("We found ",len(hdu), "HDU")

We now extract the header and the data

In [ ]:
head = hdu[0].header
data = hdu[0].data

In [ ]:
# head  is now a python dictionary
print(head.keys)

In [ ]:
print(type(data))
print(data.shape)

In [ ]:
x=757
y=304
#   this is an  example bad pixel
print(filename,x,y,data[y-1,x-1])

Now we will fix this bad pixel value by replacing it with the average of the 4 pixels below, above, right and left

In [ ]:
newvalue = (data[y-2,x-1]  + data[y,x-1] + data[y-1,x-2]  +  data[y-1,x])/4.0
print(newvalue)
data[y-1,x-1] = newvalue

Now we will write a function that loops over the whole data array (2dim) and patches each pixel which
deviates more then eps (relative number) from it's neighbors

In [ ]:
# this algorithm is too sensitive because it looks at bad pixels when it's in the neighor list, not for center
# replacement.
# we need a better algorithm:
#     -median filter?
#     - use more neighbors?
def patch_badpixels(data, eps=0.1):
    nx = data.shape[1]
    ny = data.shape[0]
    nbad = 0
    for ix in range(1,nx-1):
        for iy in range(1,ny-1):
            v1 = data[iy,ix]
            v2 = (data[iy-1,ix] + data[iy+1,ix] + data[iy,ix-1] + data[iy,ix+1])/4.0
            if abs(v1-v2) > eps:
                nbad = nbad + 1
                #print("Bad pixel",ix+1,iy+1,v1,v2)
                data[iy,ix] = v2
    return nbad
                

In [ ]:
data1 = data.copy()
nbad = patch_badpixels(data1,1000)
print("Patched ",nbad)